In [ ]:
import ee
service_account = 'googleengine@fondecyt1191543.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, 'fondecyt1191543-5bb0b5b19991.json')
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp
import geopandas as gpd
from datetime import datetime, timedelta
import  zipfile, io, shutil
import json

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Cloud functions

In [ ]:
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).select("B.*").copyProperties(image, ["system:time_start"])


In [ ]:
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

## Ubicación e imágenes pre-post incendios:

In [ ]:
cases={'Limache':{'date':datetime(2019,1,3),'loc': (-71.23338230315984, -32.98741702029078)},
        'Rodelillo':{'date':datetime(2019,11,18),'loc': (-71.58387814694001, -33.05355562738098)},
      'Miraflores':{'date':datetime(2019,11,18),'loc': (-71.52681957185263, -33.02176651781756)},
      'Rocuant':{'date':datetime(2019,12,14),'loc': (-71.58911805125635, -33.07314951329438)},
      'Quilpue':{'date':datetime(2021,1,15),'loc': (-71.41691745413526, -33.07857460989792)},
      'LagunaVerde':{'date':datetime(2022,3,30),'loc': (-71.67884530158557, -33.13923221199312)},
      'Melipilla':{'date':datetime(2022,12,15),'loc':(-71.301511,-33.800970)}}

cases={'ViñadelMar':{'date':datetime(2022,12,22),'loc':(-71.54191567666568,-33.034022110663976)}}

root='imagenesSentinel/'

In [ ]:
from shapely.geometry import Point, Polygon

for case in cases.keys():
    print('----')
    print(case)

    lon,lat=cases[case]['loc']

    bbox_file='%s/%s/%s_boundary_WGS.json'%(root,case,case)
    print(bbox_file)

    bbox=json.load(open(bbox_file,'r'))
    coords = bbox['features'][0]['geometry']['coordinates']
    aoi = ee.Geometry.Polygon(coords)
    

    #aoi = ee.Geometry.Point(lon,lat)
    cases[case]['aoi']=aoi


    date_event=cases[case]['date']
    print(date_event)
    date_pre=cases[case]['date']-timedelta(days=7)
    date_post=cases[case]['date']+timedelta(days=15)

    startDate=ee.Date(date_pre.strftime('%Y-%m-%d'))
    endDate=ee.Date(date_post.strftime('%Y-%m-%d'))

    s2=ee.ImageCollection('COPERNICUS/S2')
    #s2=ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

    #image collection
    im_coll_pre = s2.filterDate(startDate,date_event).filter(ee.Filter.eq('MGRS_TILE', '19HBD')).filterBounds(aoi)
    nim_pre=im_coll_pre.size().getInfo()
    image_pre = ee.Image(im_coll_pre.sort('system:time_start').first())
    print('Pre image: ', image_pre.get('system:id').getInfo())
    print('pre:',nim_pre)
    
    #im_coll_pre= get_s2_sr_cld_col(aoi,startDate, date_event)
    im_coll_post = s2.filterDate(date_event,endDate).filter(ee.Filter.eq('MGRS_TILE', '19HBD')).filterBounds(aoi)

    
    nim_post=im_coll_post.size().getInfo()

    image_post = ee.Image(im_coll_post.sort('system:time_start',False).first())
    print('Post image: ',image_post.get('system:id').getInfo())

    image_pre = ee.Image(im_coll_pre.sort('system:time_start').first())
    print('Pre image: ', image_pre.get('system:id').getInfo())
    print('pre:',nim_pre,'post:',nim_post)
    cases[case]['im_pre']=image_pre.get('system:id').getInfo()
    cases[case]['im_post']=image_post.get('system:id').getInfo()

                


----
ViñadelMar
imagenesSentinel//ViñadelMar/ViñadelMar_boundary_WGS.json
2022-12-22 00:00:00
Pre image:  COPERNICUS/S2/20221219T143731_20221219T144224_T19HBD
pre: 1


EEException: Element.get: Parameter 'object' is required.

## Cálculo de índices y descarga de imágenes

In [ ]:
# Write a function that computes NDVI for an image and adds it as a band
def addIndex(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
  nbr = image.normalizedDifference(['B8', 'B12']).rename('nbr')
  newBands = ee.Image([ndvi, nbr])
  image = image.addBands(newBands)
  return image



In [ ]:
import requests

#for case in cases.keys():
for case in ['LagunaVerde']:

    images={}
    path='%s/%s/'%(root,case)
    try:
        os.mkdir(path)
    except:
        pass
        
    for epoch in ['pre','post']:
        img=ee.Image(cases[case]['im_%s'%epoch]).select(['B2','B3','B4','B5','B7','B8','B12'])
        img_index=addIndex(img)
    
        uid = img.get('system:id').getInfo()
        name='S2_%s_'%epoch+uid.split('/')[-1]
        print(case,uid,name)
        
        img_clip=img_index.clip(cases[case]['aoi'])
        url=img_clip.getDownloadURL({'name':name,'bands': ['B2', 'B3', 'B4','nbr','ndvi'],'filePerBand': False})
       
        r = requests.get(url, stream=True)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall()        

        shutil.move('%s.tif'%name, '%s/%s.tif'%(path,name))
        images['%s'%epoch]=img_clip
        
    print(images)   
    dnbr=images['pre'].select('nbr').subtract(images['post'].select('nbr'))
    
    url2=dnbr.getDownloadURL({'name':'%s_dnbr'%case,'bands': ['nbr'],'filePerBand': False})
    r = requests.get(url2, stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    shutil.move('%s_dnbr.tif'%case, '%s/%s_dnbr.tif'%(path,case))
    

LagunaVerde COPERNICUS/S2/20220324T143731_20220324T144728_T19HBD S2_pre_20220324T143731_20220324T144728_T19HBD
LagunaVerde COPERNICUS/S2/20220413T143731_20220413T144923_T19HBD S2_post_20220413T143731_20220413T144923_T19HBD
{'pre': <ee.image.Image object at 0x7f99ba912b50>, 'post': <ee.image.Image object at 0x7f99beb3a970>}
